# Example code for QPARC Challenge 2022

In [163]:
# Import some modules.
# You are not allowed to import qulacs.QuantumState, nor other quantum circuit simulators.

from typing import Counter

import numpy as np
from openfermion.transforms import jordan_wigner
from qulacs import QuantumCircuit
from qulacs.gate import CZ, RY, H, Sdag
from scipy.optimize import minimize

from qparc import (
    QulacsExecutor,
    create_observable_from_openfermion_text,
    TotalShotsExceeded,
)

In [164]:
# Set up the executor, and get the problem hamiltonian.
# One must run quantum circuits always through the executor.
executor = QulacsExecutor()
fermionic_hamiltonian, n_qubits = executor.get_problem_hamiltonian()


# Process the Hamiltonian.
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

In [165]:
# One can obtain HF energy and FCI energy.
print("HF energy:", executor.hf_energy)
print("FCI energy:", executor.fci_energy)

HF energy: -2.0985459369977617
FCI energy: -2.1663874486347625


In [166]:
# Set up the ansatz


def ry_ansatz_circuit(n_qubits, depth, theta_list):
    """ry_ansatz_circuit
    Returns Ry ansatz circuit.

    Args:
        n_qubits:
            The number of qubit used
        depth:
            Depth of the circuit.
        theta_list:
            Rotation angles.
    Returns:
        circuit:
            Resulting Ry ansatz circuit.
    """
    circuit = QuantumCircuit(n_qubits)
    params_id = 0
    for d in range(depth):
        for i in range(n_qubits):
            circuit.add_gate(
                RY(i, theta_list[params_id]),
            )
            params_id += 1
        for i in range(n_qubits // 2):
            circuit.add_gate(CZ(2 * i, 2 * i + 1))
        for i in range(n_qubits // 2 - 1):
            circuit.add_gate(CZ(2 * i + 1, 2 * i + 2))
    for i in range(n_qubits):
        circuit.add_gate(RY(i, theta_list[params_id]))
        params_id += 1

    return circuit

In [167]:
def get_terms_and_measurement_circuits(observable):
    """get_terms_and_measurement_circuits
    Returns basis-rotation circuits for measurement, along with the corresponding terms.

    Args:
        observable:
            The observable to be measured.
    Returns:
        pauli_coef:
            List of coefficients.
        pauli_target:
            List of targetted qubits.
        pauli_gate:
            List of circuits for basis-rotation.
    """
    pauli_coef = []
    pauli_target = []
    pauli_gate = []
    n_qubits = observable.get_qubit_count()
    for i_term in range(observable.get_term_count()):
        term = observable.get_term(i_term)
        pauli_coef.append(term.get_coef())
        target_list = term.get_index_list()
        pauli_target.append(target_list)
        id_list = term.get_pauli_id_list()
        circuit = QuantumCircuit(n_qubits)
        for target, id in zip(target_list, id_list):
            if id == 1:
                circuit.add_gate(H(target))
            elif id == 2:
                circuit.add_gate(Sdag(target))
                circuit.add_gate(H(target))
            elif id == 3:
                pass
            else:
                raise Exception(f"Operator {target, id} not supported")
        pauli_gate.append(circuit)
    return pauli_coef, pauli_target, pauli_gate

In [168]:
def get_energy(theta_list, n_shots, depth, state, hamiltonian):
    """get_energy
    Returns the evaluated energy

    Args:
        theta_list:
            The parameters
        n_shots:
            The number of shots used to evaluate each term.
        depth:
            The depth of the ansatz
        state:
            The integer that defines the initial state in the computational basis.
        hamiltonian:
            The Hamiltonian to be evaluated.
    Returns:
        ret:
            The evaluated energy.
    """
    pauli_coef, pauli_target, pauli_gate = get_terms_and_measurement_circuits(
        hamiltonian
    )
    n_qubits = hamiltonian.get_qubit_count()
    circuit = ry_ansatz_circuit(n_qubits=n_qubits, depth=depth, theta_list=theta_list)
    ret = 0
    for coef, target, gate in zip(pauli_coef, pauli_target, pauli_gate):
        if target:
            counts = Counter(
                executor.sampling(
                    [circuit, gate],
                    state_int=state,
                    n_qubits=n_qubits,
                    n_shots=n_shots,
                )
            )
            for sample, count in counts.items():
                binary = np.binary_repr(sample).rjust(n_qubits, "0")
                measurement = np.product(
                    [-1 if binary[n_qubits - t - 1] == "1" else 1 for t in target]
                )
                ret += coef * measurement * count / n_shots
        else:
            ret += coef

    return ret.real

In [169]:
# not used in the example
def get_gradient(theta_list, n_shots, depth, state, hamiltonian):
    """get_gradient
    Returns the evaluated gradient of the energy in the parameter space.

    Args:
        theta_list:
            The parameters
        n_shots:
            The number of shots used to evaluate each term.
        depth:
            The depth of the ansatz
        state:
            The integer that defines the initial state in the computational basis.
        hamiltonian:
            The Hamiltonian to be evaluated.
    Returns:
        np.array(g):
            The gradient of the energy in the parameter space.
    """
    g = []

    param_dim = len(theta_list)
    for i in range(param_dim):
        shift = np.zeros(param_dim)
        shift[i] = 0.5 * np.pi
        gi = 0.5 * (
            get_energy(
                theta_list=theta_list + shift,
                n_shots=n_shots,
                depth=depth,
                state=state,
                hamiltonian=hamiltonian,
            )
            - get_energy(
                theta_list=theta_list - shift,
                n_shots=n_shots,
                depth=depth,
                state=state,
                hamiltonian=hamiltonian,
            )
        )
        g.append(gi)
    return np.array(g)

In [170]:
# Define input settings
n_shots = 10000
initial_state = 0b00001111
depth = 2

In [171]:
# Define functions to be used in the optimization process.
def cost(theta_list, state=initial_state):
    ret = get_energy(
        theta_list=theta_list,
        n_shots=n_shots,
        depth=depth,
        state=state,
        hamiltonian=qulacs_hamiltonian,
    )
    # executor.current_value will be used as the final result when the number of shots reach the limit,
    # so set the current value as often as possible, if you find a better energy.
    if ret < executor.current_value:
        executor.current_value = ret
    return ret


def grad(theta_list):
    ret = get_gradient(
        theta_list=theta_list,
        n_shots=n_shots,
        depth=depth,
        state=initial_state,
        hamiltonian=qulacs_hamiltonian,
    )
    return ret


def callback(theta_list):
    print("current val", executor.current_value)

In [172]:
# This block runs the VQE algorithm.
# The result will be finalized when executor.record_result() is called.

init_theta_list = np.random.random(n_qubits * (depth + 1)) * 0.01
method = "Powell"
options = {"disp": True, "maxiter": 10000}
try:
    opt = minimize(
        cost,
        init_theta_list,
        method=method,
        # jac=grad,
        options=options,
        callback=callback,
    )
except TotalShotsExceeded as e:
    print(e)
    executor.record_result()
else:
    print("Optimization finished without reaching the shot limit")
    executor.record_result()

The number of total shots exceeded the limit.

############## Result ##############
Resulting energy = -2.1030561152860785
total_shots = 100000000
------------------------------------
FCI energy = -2.1663874486347625
HF energy = -2.0985459369977617
####################################



In [173]:
# This block is for final evaluation.
# The average accuracy will be evaluated.

executor.reset()
method = "Powell"
options = {"disp": True, "maxiter": 10000}
for _ in range(10):
    init_theta_list = np.random.random(n_qubits * (depth + 1)) * 0.01
    try:
        opt = minimize(
            cost,
            init_theta_list,
            method=method,
            # jac=grad,
            options=options,
            callback=callback,
        )
    except TotalShotsExceeded as e:
        print(e)
        executor.record_result(verbose=False)
    else:
        print("Optimization finished without reaching the shot limit")
        executor.record_result(verbose=False)
executor.evaluate_final_result()

The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.
The number of total shots exceeded the limit.

############## Final Result ##############
Average energy: -2.103716804697547
Average accuracy: 0.06267064393721525
------------------------------------
FCI energy = -2.1663874486347625
HF energy = -2.0985459369977617
####################################
